## Imports



In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

#Facebook Prophet
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric

## Data-Split

In [12]:
df=pd.read_csv('https://storage.googleapis.com/energy_consumption_903/df_all_avg_v3.csv')
def split_data(filename):
#     print('Input precise path for data including extension')
#     filename = input()
    fulldata = pd.read_csv(filename)
    fulldata['DateTime'] = pd.to_datetime(fulldata['DateTime'])
    train_data = fulldata[(fulldata['DateTime'] >= '2012-01-01') & (fulldata['DateTime'] < '2014-01-01')].reset_index(drop = True)
    validation_data = fulldata[(fulldata['DateTime'] >= '2014-01-01') & (fulldata['DateTime'] < '2014-02-01')].reset_index(drop = True)
    test_data = fulldata[(fulldata['DateTime'] >= '2014-02-01') & (fulldata['DateTime'] < '2014-03-01')].reset_index(drop = True)
    return train_data, validation_data, test_data

## Data-Processing for global models

In [49]:
tdata, vdata, testd=split_data('https://storage.googleapis.com/energy_consumption_903/df_all_avg_v3.csv')

# # not for global average
# train_data.drop(columns='Unnamed: 0',inplace=True)
# validation_data.drop(columns='Unnamed: 0',inplace=True)
# test_data.drop(columns='Unnamed: 0',inplace=True)

# add val and train for prophet
pdata=pd.concat([tdata,vdata],axis=0).reset_index(drop=True)
pdata

,DateTime,KWH/hh,tariff,Acorn_Group
0,2012-01-01 00:00:00,0.471778,Mixed,A
1,2012-01-01 00:30:00,0.483222,Mixed,A
2,2012-01-01 01:00:00,0.457222,Mixed,A
3,2012-01-01 01:30:00,0.380556,Mixed,A
4,2012-01-01 02:00:00,0.452889,Mixed,A
...,...,...,...,...
620705,2014-01-31 21:30:00,0.259288,Mixed,Q
620706,2014-01-31 22:00:00,0.237047,Mixed,Q
620707,2014-01-31 22:30:00,0.231066,Mixed,Q
620708,2014-01-31 23:00:00,0.224832,Mixed,Q


In [50]:
# group all for dumb models
df5=pdata.loc[:,['Acorn_Group','KWH/hh']]
train_df=df5.groupby(by=df5.index).mean()
train_df=data_gla.reset_index()

,Acorn_Group,KWH/hh
0,A,0.471778
1,A,0.483222
2,A,0.457222
3,A,0.380556
4,A,0.452889
...,...,...
620705,Q,0.259288
620706,Q,0.237047
620707,Q,0.231066
620708,Q,0.224832


In [48]:
train_df

,level_0,index,KWH/hh
0,0,0,0.447229
1,1,1,0.392184
2,2,2,0.361908
3,3,3,0.340252
4,4,4,0.289180
...,...,...,...
25291,25291,25291,0.259288
25292,25292,25292,0.237047
25293,25293,25293,0.231066
25294,25294,25294,0.224832


## Optional Parameters

### Holidays

In [14]:
holidays=pd.read_csv(os.path.join(os.getcwd(),'../raw_data/uk_bank_holidays.csv'))
holidays.rename(columns={'Type':'holiday','Bank holidays':'ds'},inplace=True)
holidays.loc[:,'ds']=pd.to_datetime(holidays['ds'],format="%d/%m/%Y")
holidays.head()

,holiday,ds
0,Boxing Day,2012-12-26
1,Christmas Day,2012-12-25
2,Summer bank holiday,2012-08-27
3,Queen's Diamond Jubilee,2012-05-06
4,Spring bank holiday,2012-04-06


### Weather

In [16]:
wd=pd.read_csv('https://storage.googleapis.com/weather-data-processed-for-le-wagon/cleaned_weather_hourly_darksky.csv')
wd_filt=wd[['time','temperature','windSpeed','precipType_rain']]
wd_filt.rename(columns={'time':'DateTime'},inplace=True)
temp = wd_filt['temperature'].interpolate(method='linear')
# wind = wd_filt['windSpeed'].interpolate(method='linear')
# rain = wd_filt['precipType_rain'].interpolate(method='linear')
df[['DateTime']].merge(wd_filt,on='DateTime',how='inner')

/tmp/ipykernel_23537/3885505480.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wd_filt.rename(columns={'time':'DateTime'},inplace=True)


## Train

In [ ]:
data_gla.rename(columns={"DateTime": "ds", "KWH/hh": "y"},inplace=True)
data_gla_regressor=data_gla
data_gla_regressor['temp']=temp

def (df_train,wd_train,holidays):
    m = Prophet(holidays=holidays,changepoint_prior_scale=0.01).add_regressor('temp', prior_scale=0.5, mode='multiplicative')
    m.fit(data_gla_regressor)
return m    